# Import

In [4]:
import pandas as pd
import json
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
master_port = pd.read_csv('ports_master.csv').iloc[:,1:]
# port = master_port[['LOCODE','draft0_r','draft1_r','draft2_r','draft3_r']].itertuples(index=False)
PORTS = list(master_port[['LOCODE','draft0_r','draft1_r','draft2_r','draft3_r','due_0', 'due_1', 'due_2', 'due_3', 'thc/teu','tranship/teu']].itertuples(index=False, name=None))
# Port = namedtuple('port',['LOCODE','draft0','draft1','draft2','draft3','due_0', 'due_1', 'due_2', 'due_3', 'thc/teu','tranship/teu'])
# PORT[]
PORTS [55:61],len(PORTS)
ves_size = [170,275,335,390]
regions_cod = ('AFR','EUR','ELT','WLT','MDE','NAM','OCE')
for reg in regions_cod:
    sc = (reg, 20000, 20000, 20000, 20000, 0, 0, 0, 0, 0, 999)
    PORTS.append(sc)

In [ ]:
pv = [(p,v) for p in range(len(PORTS)) for v in range(4) ]

# Port berth relax

In [7]:
with open('Solution_relaxed.json') as json_file:
    sol1 =json.load(json_file)
len(sol1),type(sol1)

(1, dict)

In [8]:
res = len(sol1['CPLEXSolution']['variables'])
res_df = pd.DataFrame(sol1['CPLEXSolution']['variables'])[['name','value']]
res_df

,name,value
0,ves_deploy_2_1,1.8929173089016205
1,ves_deploy_2_3,1.2044524728213402
2,ves_deploy_7_2,2.2993937131388114
3,ves_deploy_14_0,6.636117415701456
4,ves_deploy_14_1,7.628014160965491
...,...,...
5181,relax_route_cap_407,384909.0
5182,relax_route_cap_413,4627.0
5183,relax_route_cap_425,149397.0
5184,relax_route_cap_429,8580.0


In [20]:
val_all = res_df.name.tolist()
sel_val = [s for s in val_all if 'port_calls_57' in s]
res_df[res_df.name.isin(sel_val)]
#look 57, 47,3,44,64,50,45

,name,value
3510,port_calls_57_0,462.8828452703367
3511,port_calls_57_1,78.08882561837817
3512,port_calls_57_2,33.42881849493125
3513,port_calls_57_3,15.285502062046238


In [22]:
print(PORTS[57])
port_call = res_df[res_df.name.isin(sel_val)].value.tolist()


('SGSIN', 1700, 1375, 0, 17160, 4018, 5768, 8268, 8268, 65, 42)


In [38]:
PORTS[57][5:9]

(4018, 5768, 8268, 8268)

In [13]:
sol1['CPLEXSolution']['variables'][1]

{'index': '11', 'name': 'ves_deploy_2_3', 'value': '1.1710860587147227'}

# Ves Cap Relaxed

In [9]:
with open('Solution_relaxed.json') as json_file:
    sol1 =json.load(json_file)
print(len(sol1),type(sol1))
res = len(sol1['CPLEXSolution']['variables'])
res_df = pd.DataFrame(sol1['CPLEXSolution']['variables'],)[['name','value']]
res_df['value'] = res_df['value'].astype(float)
res_df

1 <class 'dict'>


,name,value
0,ves_deploy_2_1,1.892917
1,ves_deploy_2_3,1.204452
2,ves_deploy_7_2,2.299394
3,ves_deploy_14_0,6.636117
4,ves_deploy_14_1,7.628014
...,...,...
5181,relax_route_cap_407,384909.000000
5182,relax_route_cap_413,4627.000000
5183,relax_route_cap_425,149397.000000
5184,relax_route_cap_429,8580.000000


In [134]:
val_all = res_df.name.tolist()
sel_val = [s for s in val_all if 'relax_route_cap' in s]
cap_relax = res_df[res_df.name.isin(sel_val)]
cap_relax.sort_values(by='value',ascending=False)

,name,value
5181,relax_route_cap_407,384909.0
5183,relax_route_cap_425,149397.0
5184,relax_route_cap_429,8580.0
5185,relax_route_cap_430,7563.0
5182,relax_route_cap_413,4627.0


In [107]:
# SGSIN Berth occupancy
#63 TWKHH
port_sel = 15
print(PORTS[port_sel])
val_all = res_df.name.tolist()
sel_val = [s for s in val_all if f'port_calls_{port_sel}' in s]
selcall = res_df[res_df.name.isin(sel_val)]
print(selcall)
port_oc = [a * b for a, b in zip(selcall.value.tolist(),ves_size)]
port_oc,PORTS[port_sel][:5],sum(port_oc)

('HKHKG', 3060, 0, 1005, 5070, 8309, 11809, 16809, 16809, 128, 46)
                 name      value
3800  port_calls_15_0  18.000000
3801  port_calls_15_2   4.115259
3802  port_calls_15_3  12.042021


([3060.0, 1131.6961337074486, 4034.07716918118],
 ('HKHKG', 3060, 0, 1005, 5070),
 8225.773302888629)

In [14]:
res_df[res_df.name.isin(lst.values())]


,name,value
3800,port_calls_15_0,18.0
3801,port_calls_15_2,4.115258668027086
3802,port_calls_15_3,12.042021400540836


# Util

In [11]:
port_sel = 15
val_all = res_df.name.tolist()
print(PORTS[port_sel])
# Port = namedtuple('port',['LOCODE','draft0','draft1','draft2','draft3','due_0', 'due_1', 'due_2', 'due_3', 'thc/teu','tranship/teu'])
lst = {(port_sel,v):s for v in range(4) 
            for s in val_all 
            if f'port_calls_{port_sel}_{v}' in s}
port_call = []
for v in range(4):
    if (port_sel,v) in lst:
        port_call.append(res_df[res_df.name == lst[(port_sel,v)]].value.sum())
    else:
        port_call.append(0)

port_call

('HKHKG', 3060, 0, 1005, 5070, 8309, 11809, 16809, 16809, 128, 46)


['18.0', 0, '4.115258668027086', '12.042021400540836']

In [18]:
for p in range(len(PORTS)):
        port_berth = PORTS[p][1:5]
port_berth 

(20000, 20000, 20000, 20000)

In [52]:
def get_berth_occupancy(df,verbose_err = False):
    vessel_len = [175, 275,335,390]
    oc_rate = {}
    val_all = df.name.tolist()
    lst = {(p,v):s for p in range(len(PORTS)) for v in range(4) for s in val_all if f'port_calls_{p}_{v}' in s}
    print(lst.keys())
    for p in range(len(PORTS)):
        port_berth = PORTS[p][1:5]
        port_call = []
        for v in range(4):
            # if p==15 :print("15",lst(p,v) in lst)
            if (p,v) in lst.keys():
                port_call.append(df[df.name == lst[(p,v)]].value.sum())
            else:
                port_call.append(0)
        port_used = [a * b for a, b in zip(port_call,ves_size)]
        port_oc = sum(port_used) / sum(port_berth)
        if (port_oc==0 or port_oc>7) and verbose_err:
            print(p,port_oc,PORTS[p][0],port_used,port_call,port_berth )
        oc_rate[p] = (PORTS[p][0],port_oc)
    return oc_rate
# berth_occupancy(res_df)


In [ ]:
def get_throughput(df,annual=False,toint = True):
    """
    Throughput only
    """
    val_all = df.name.tolist()
    lst = [s for s in val_all if 'port' in s]
    _df = df[res_df.name.isin(lst)].copy()
    print(_df.shape)
    _df['Port_ind']= _df.name.apply(lambda x: x[-2:].replace('_',''))
    # thrp_all['Port_ind'] = thrp_all['Port_ind'].replace('_','')
    _df['Port_name']= _df.Port_ind.apply(lambda x: PORTS[int(x)][0])
    _df = _df.sort_values(by='value',ascending=False)
    if toint:
        _df.value = _df.value.astype(int)
    _df = _df[['Port_ind','Port_name','value']]
    if annual:
        _df['Annually'] = _df.value.apply(lambda x: x*50/1e6)
    return _df.reset_index(drop=True)

In [65]:
def get_ope_param(df,param):
    """
    ope params included
        port_load, port_unload,
        imp,exp,port_trans
    """
    valid_name = ['imp','exp', 'port_load', 'port_unload', 'port_trans']
    if param not in valid_name:
        print('Invalid Parameter')
        return None

    val_all = df.name.tolist()
    lst = [s for s in val_all if param in s]
    _df = df[res_df.name.isin(lst)].copy()
    print(_df.shape)
    _df['Port_ind']= _df.name.apply(lambda x: x.split('_')[-2])
    return _df
    # thrp_all['Port_ind'] = thrp_all['Port_ind'].replace('_','')
    # _df['Port_name']= _df.Port_ind.apply(lambda x: PORTS[int(x)][0])
    # _df = _df.sort_values(by='value',ascending=False)
    # if toint:
    #     _df.value = _df.value.astype(int)
    # _df = _df[['Port_ind','Port_name','value']]
    # if annual:
    #     _df['Annually'] = _df.value.apply(lambda x: x*50/1e6)
    # return _df.reset_index(drop=True)


In [ ]:
get_ope_param(res_df,'ope')

In [63]:
# [s for s in val_all if 'ves_deploy' in s]
get_throughput(res_df)

(877, 2)


,Port_ind,Port_name,value
0,12,CNYTN,3285879
1,72,NAM,3000881
2,3,CNQZH,2711125
3,71,MDE,2015307
4,22,INCOK,1352487
...,...,...,...
872,23,INVTZ,0
873,17,IDSUB,0
874,23,INVTZ,0
875,17,IDSUB,0


# Integer Read
 => Throughput

 => Transhipment

 => Occupancy

In [50]:
with open('Solution_Master_int.json') as json_file:
    sol1 =json.load(json_file)
print(len(sol1),type(sol1))
res = len(sol1['CPLEXSolution']['variables'])
res_df = pd.DataFrame(sol1['CPLEXSolution']['variables'],)[['name','value']]
res_df['value'] = res_df['value'].astype(float)
res_df

1 <class 'dict'>


,name,value
0,ves_deploy_7_1,2.0
1,ves_deploy_16_0,1.0
2,ves_deploy_24_3,1.0
3,ves_deploy_35_0,3.0
4,ves_deploy_37_0,1.0
...,...,...
5012,port_thrp_69,303102.0
5013,port_thrp_70,137152.0
5014,port_thrp_71,2015308.0
5015,port_thrp_72,3000881.0


In [58]:
get_throughput('ves_deploy',res_df)

NameError: name 'get_throughput' is not defined

In [ ]:
get_all_portval('ves_deploy',res_df)

In [ ]:
val_all = res_df.name.tolist()
sel_val = [s for s in val_all if 'relax_route_cap' in s]
cap_relax = res_df[res_df.name.isin(sel_val)]
cap_relax.sort_values(by='value',ascending=False)

In [ ]:
with open('Solution_relaxed.json') as json_file:
    sol1 =json.load(json_file)
print(len(sol1),type(sol1))
res = len(sol1['CPLEXSolution']['variables'])
res_df = pd.DataFrame(sol1['CPLEXSolution']['variables'],)[['name','value']]
res_df['value'] = res_df['value'].astype(float)
res_df

In [ ]:
CHNBG SGSIN CNSHG MYPKG KRPUS HKHKG CHYTN IDJKT CNSHK XM